In [1]:
#| label: mtrFig2jn

# Prepare Python environment

import scipy.io as sio
import numpy as np
from pathlib import Path


from contextlib import contextmanager
import sys, os
from pathlib import Path

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

import os
from pathlib import Path

def find_myst_yml_directories(start_dir=None):
    """
    Recursively search for directories containing myst.yml file.
    
    Args:
        start_dir (str or Path): Starting directory (defaults to current directory)
    
    Returns:
        list: List of full paths to directories containing myst.yml
    """
    if start_dir is None:
        start_dir = Path.cwd()
    else:
        start_dir = Path(start_dir)
    
    myst_dirs = []
    
    def _search_directory(current_dir):
        # Check if myst.yml exists in current directory
        myst_file = current_dir / "myst.yml"
        if myst_file.exists():
            myst_dirs.append(str(current_dir.resolve()))
            # Don't search subdirectories if we found myst.yml here
            return
        
        # Recursively search all subdirectories
        for item in current_dir.iterdir():
            if item.is_dir():
                try:
                    _search_directory(item)
                except (PermissionError, OSError):
                    # Skip directories we can't access
                    continue
    
    _search_directory(start_dir)
    return myst_dirs

def find_myst_yml_directories_upwards(start_dir=None):
    """
    Search for myst.yml in current directory, if not found go to parent and repeat.
    
    Args:
        start_dir (str or Path): Starting directory (defaults to current directory)
    
    Returns:
        str or None: Full path of directory containing myst.yml, or None if not found
    """
    if start_dir is None:
        current_dir = Path.cwd()
    else:
        current_dir = Path(start_dir)
    
    # Keep going up until we reach the filesystem root
    while current_dir != current_dir.parent:  # Stop at root
        myst_file = current_dir / "myst.yml"
        if myst_file.exists():
            return str(current_dir.resolve())
        
        # Move to parent directory
        current_dir = current_dir.parent
    
    return None

with suppress_stdout():
    repo_path = Path(find_myst_yml_directories_upwards())
    print(repo_path)
    data_req_path = repo_path / "binder" / "data_requirement.json"
    data_path = repo_path / "data"
    dataset_path = data_path / "qmrlab-mooc"
    
data_dir = dataset_path / "06-MT-02-MTR" / "06-MT-02-MTR" 
data_file = "fig4.mat"

#Load either archived or generated plot variables
mat_contents = sio.loadmat(data_dir / data_file)

MTRs = mat_contents["MTRs"][0]
T1f = mat_contents["T1_range"][0]
T1_true = mat_contents["T1_true"][0]

x_T1_true = np.ones(len(MTRs))*T1_true[0]
y_T1_true = np.linspace(MTRs[0]*0.95, MTRs[-1]*1.05, num=len(MTRs))

# Plot Figure 1

# Module imports

import matplotlib.pyplot as plt
import plotly as py
import plotly.graph_objs as go
import numpy as np
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
config={'showLink': False, 'displayModeBar': False}

init_notebook_mode(connected=True)

# PYTHON CODE

init_notebook_mode(connected=True)
# The polling here is to ensure that plotly.js has already been loaded before
# setting display alignment in order to avoid a race condition.


data = [
    go.Scatter(
        x=T1f,
        y=MTRs,
        name = "Protocol: Brown 2013 (Philips)",
        text = 'N/A',
        hoverinfo = 'y'
        ),
    go.Scatter(
        x=x_T1_true,
        y=y_T1_true,
        name = "True T1f",
        text = 'N/A',
        hoverinfo = 'y',
        line = dict(shape = 'linear', color = 'rgb(0, 0, 0)', width = 2, dash = 'dash'),
        ),
        
]


layout = go.Layout(
    width=600,
    height=600,
    margin=go.layout.Margin(
        l=100,
        r=80,
        b=100,
        t=130,
    ),
    annotations=[
        dict(
            x=-0.15,
            y=0.50,
            showarrow=False,
            text='MTR',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            textangle=-90,
            xref='paper',
            yref='paper'
        ),
        dict(
            x=0.5,
            y=-0.15,
            showarrow=False,
            text='T1f (s)',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            textangle=0,
            xref='paper',
            yref='paper'
        ),
    ],
    xaxis=dict(
        showgrid=True,
        gridcolor='rgb(169,169,169)',
        linecolor='black',
        linewidth=2
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor='rgb(169,169,169)',
        linecolor='black',
        linewidth=2,
        range = [0.36, 0.5]
    ),
    legend=dict(
        x=0.25,
        y=1.2,
        traceorder='normal',
        font=dict(
            family='Times New Roman',
            size=12,
            color='#000'
        ),
        bordercolor='#000000',
        borderwidth=2
    )
)

fig = dict(data=data, layout=layout)


iplot(fig, filename = 'fig4.html', config = config)
